In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
import json
import pickle
from replay_buffer.ber import BlockReplayBuffer
from ray.rllib.utils.replay_buffers import ReplayBuffer
from replay_buffer.mpber import MultiAgentPrioritizedBlockReplayBuffer
from ray.rllib.utils.replay_buffers.multi_agent_prioritized_replay_buffer import MultiAgentPrioritizedReplayBuffer

/home/seventheli/conda/envs/hpc/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def generate_configs(base_path):
    configs = {}
    for env in os.listdir(base_path):
        env_path = os.path.join(base_path, env)
        if os.path.isdir(env_path):
            configs[env] = {}
            for experiment in os.listdir(env_path):
                experiment_path = os.path.join(env_path, experiment)
                if os.path.isdir(experiment_path):
                    config_file = os.path.join(experiment_path, experiment + "_config.pyl")
                    if os.path.exists(config_file):
                        with open(config_file, "rb") as f:
                            configs[env][experiment] = pickle.load(f)
                    else:
                        print("config not existing for %s in %s" % (experiment, env))
    return configs

base_path = "/home/seventheli/JADE_checkpoints/"
configs = generate_configs(base_path)

config not existing for DQN_ER_SpaceInvadersNoFrameskip-v4_511703 in SpaceInvaders


In [3]:
pd.concat([pd.read_csv("jade_er.csv"), pd.read_csv("jade_per.csv")]).to_csv("./jade_all.csv", index=False)

In [4]:
data_path = "./jade_all.csv"

In [5]:
mapper = {
    ReplayBuffer: "ER",
    BlockReplayBuffer: "BER",
    MultiAgentPrioritizedReplayBuffer: "PER",
    MultiAgentPrioritizedBlockReplayBuffer: "PBER",
}

In [6]:
data = pd.read_csv(data_path)
data['env'] = data['exp'].str.split('_').apply(lambda x: x[2].replace("NoFrameskip-v4", ""))
data["buffer"] = data.apply(lambda x: configs[x["env"]][x["exp"]].get("replay_buffer_config")["type"], axis=1)
data["buffer"] = data["buffer"].apply(lambda x: mapper[x])
data["size"] = data.apply(lambda x: configs[x["env"]][x["exp"]].get("replay_buffer_config").get("sub_buffer_size", 1), axis=1)
data.head()
data.to_csv("./jade_all.csv", index=False)

In [7]:
max_exp = data.groupby(['env', 'buffer', 'size'])['exp'].apply(lambda x: x.value_counts().idxmax())

filtered_data = data[data['exp'].isin(max_exp.values)].reset_index(drop=True)
# Group by 'env', 'size', 'buffer' and aggregate the required metrics
summary = filtered_data.groupby(['env', 'size', 'buffer']).agg({
    'episode_reward_mean': 'last',  # Last episode reward mean
    'num_env_steps_sampled': 'sum', # Total number of data points
    'time_total_s': 'last',         # Total time
    'exp': 'unique'                 # Experiment names
}).reset_index()

# Add a column to indicate the number of unique experiments for each group
summary['num_experiments'] = summary['exp'].apply(len)

# Verify if there's only 1 experiment for each group
summary['single_experiment'] = summary['num_experiments'] == 1

# Display the summary
summary.single_experiment.value_counts()

single_experiment
True    75
Name: count, dtype: int64

In [8]:
filtered_data.to_csv("jade_all_filtered.csv", index=False)

In [9]:
# Correctly compute the baseline difference for each environment
baseline_diff_corrected = filtered_data.groupby('env').apply(
    lambda x: x['episode_reward_mean'].max() - x['episode_reward_mean'].min()
)

# Using the corrected baseline difference to detect anomalies

# Function to detect anomalies based on the corrected baseline
def detect_anomalies_baseline(exp_data, env_baseline_diff):
    exp_diff = abs(exp_data['episode_reward_mean'].max() - exp_data['episode_reward_mean'].min())
    return exp_diff < abs(env_baseline_diff * 0.20)

anomalies_baseline_corrected = []

# Check each experiment in the data
for env, env_data in filtered_data.groupby('env'):
    env_baseline_diff = baseline_diff_corrected[env]
    for exp, exp_data in env_data.groupby('exp'):
        if detect_anomalies_baseline(exp_data, env_baseline_diff):
            anomalies_baseline_corrected.append(exp)

In [10]:
filtered_data = filtered_data[~filtered_data['exp'].isin(anomalies_baseline_corrected)].reset_index(drop=True)

In [11]:
filtered_data.to_csv("jade_all_filtered.csv", index=False)